# Deep and Reinforcement Learning 2024/2025 (M.IA003), FEUP/FCUP

## Deep Learning Project 
## **Develop deep learning discriminative and generative models, applied to the context of “deep fakes”**

work done by:
- Michal Kowalski up
- Pedro Pereira up
- Pedro Azevedo up201905966

## 1.1) Import Necessary

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import os

In [ ]:
# Define dataset directories
data_dir = "data"
real_dir = os.path.join(data_dir, "real")
fake_dir = os.path.join(data_dir, "fake")

## 1.2) Basic EDA

In [ ]:
from _eda import base_stats

base_stats(real_dir, fake_dir)

## 1.3) File Standardization 

In [ ]:
from _preprocess import standardize_files

standardize_files('data/real', (128, 128), '128')
standardize_files('data/fake', (128, 128), '128')

In [ ]:
real_dir = os.path.join(data_dir, "real_normalized_128")
fake_dir = os.path.join(data_dir, "fake_normalized_128")
base_stats(real_dir, fake_dir)

## 1.4) Baseline

In [ ]:
from _model import build_gan, train_gan

d, g, gan = build_gan()

In [ ]:
import tensorflow as tf
import glob

batch_size = 256
# Get all image file paths
image_paths = glob.glob("./data/real_normalized_128/*.jpg")
# Limit to 5,000 images
# image_paths = image_paths[:5000]


# Define the preprocessing function
def load_and_preprocess(image_path):
    # Read the image file
    image = tf.io.read_file(image_path)

    # Decode the JPEG image
    image = tf.image.decode_jpeg(image, channels=3)

    # Normalize the image to the range [-1, 1]
    image = (tf.cast(image, tf.float32) / 127.5) - 1.0

    return image

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size//2, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

# # Inspect the dataset

batches = dataset.take(39)
for batch in batches:
    print("Batch shape:", batch.shape)  # Should be (64, 128, 128, 3)


In [ ]:
gan.summary()

In [ ]:
train_gan(dataset, d, g, gan, epochs=100, batch_size=256)

In [ ]:
gan.save('my_gan.keras')

In [ ]:
from tensorflow.keras.models import load_model
saved_model = load_model('./my_gan.keras')
saved_model.summary()

In [ ]:
g = saved_model.layers[1]
g.summary()

In [ ]:
d = saved_model.layers[2]
gan = saved_model

In [ ]:
train_gan(dataset, d, g, gan, epochs=395, batch_size=256)